<a href="https://colab.research.google.com/github/AN-SAFAWAT/File_ingestion-/blob/main/data_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    #df.drop('Unnamed: 0',axis=1)
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


In [2]:
%%writefile file.yaml
file_type: csv
dataset_name: DrDoSDNS
file_name: DrDoS_DNS
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - flow_id
    - source_ip
    - source_port

Writing file.yaml


In [3]:
import testutility as tu

In [4]:
config_data=tu.read_config_file('file.yaml')
config_data

{'file_type': 'csv',
 'dataset_name': 'DrDoSDNS',
 'file_name': 'DrDoS_DNS',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['flow_id', 'source_ip', 'source_port']}

In [5]:
config_data['columns']

['flow_id', 'source_ip', 'source_port']

In [10]:
len(config_data['columns'])

3

In [11]:
#from pandas.core.generic import config
import pandas as pd

file_type=config_data['file_type']
url='/content/'+config_data['file_name']+f'.{file_type}'

df_yaml=pd.read_csv(url)
df_yaml.drop('Unnamed: 0',axis=1,inplace=True)
df_yaml

<ipython-input-11-428a7e329ef2>:7: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df_yaml=pd.read_csv(url)


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,172.16.0.5-192.168.50.1-634-60495-17,172.16.0.5,634,192.168.50.1,60495,17,2018-12-01 10:51:39.813448,28415,97,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,DrDoS_DNS
1,172.16.0.5-192.168.50.1-60495-634-17,192.168.50.1,634,172.16.0.5,60495,17,2018-12-01 10:51:39.820842,2,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,DrDoS_DNS
2,172.16.0.5-192.168.50.1-634-46391-17,172.16.0.5,634,192.168.50.1,46391,17,2018-12-01 10:51:39.852499,48549,200,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,DrDoS_DNS
3,172.16.0.5-192.168.50.1-634-11894-17,172.16.0.5,634,192.168.50.1,11894,17,2018-12-01 10:51:39.890213,48337,200,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,DrDoS_DNS
4,172.16.0.5-192.168.50.1-634-27878-17,172.16.0.5,634,192.168.50.1,27878,17,2018-12-01 10:51:39.941151,32026,200,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,DrDoS_DNS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85542,172.16.0.5-192.168.50.1-792-54540-17,172.16.0.5,792,192.168.50.1,54540,17,2018-12-01 11:06:12.838194,51,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,DrDoS_DNS
85543,172.16.0.5-192.168.50.1-793-48632-17,172.16.0.5,793,192.168.50.1,48632,17,2018-12-01 11:06:12.838292,2,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,DrDoS_DNS
85544,172.16.0.5-192.168.50.1-564-14935-17,172.16.0.5,564,192.168.50.1,14935,17,2018-12-01 11:06:12.838346,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,DrDoS_DNS
85545,172.16.0.5-192.168.50.1-564-7820-17,172.16.0.5,564,192.168.50.1,7820,17,2018-12-01 11:06:12.838395,46,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,DrDoS_DNS


column header validation.

In [12]:
tu.col_header_val(df_yaml,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['fwd_iat_std', 'flow_bytes_s', 'idle_min', 'urg_flag_count', 'average_packet_size', 'bwd_packet_length_min', 'idle_std', 'simillarhttp', 'total_backward_packets', 'fwd_avg_packets_bulk', 'fwd_header_length_1', 'active_mean', 'destination_port', 'subflow_bwd_packets', 'flow_iat_std', 'fwd_packet_length_max', 'fwd_iat_total', 'bwd_iat_min', 'total_length_of_bwd_packets', 'bwd_packet_length_std', 'active_std', 'flow_duration', 'fwd_packet_length_min', 'flow_iat_mean', 'fwd_iat_mean', 'fwd_header_length', 'fwd_avg_bulk_rate', 'fwd_iat_min', 'bwd_iat_std', 'init_win_bytes_backward', 'bwd_avg_packets_bulk', 'bwd_header_length', 'ece_flag_count', 'protocol', 'init_win_bytes_forward', 'max_packet_length', 'subflow_fwd_bytes', 'rst_flag_count', 'flow_iat_min', 'active_max', 'bwd_urg_flags', 'fwd_packet_length_std', 'destination_ip', 'fwd_psh_flags', 'idle_max', 'bwd_psh_flags', 'min_packet_length', 

0

save the dataframe as output.txt with separator (|)

In [9]:
df_yaml.to_csv(r'/content/output.txt', sep = '|', index=False)